In [1]:
from lcls_live.datamaps.tabular import TabularDataMap
from lcls_live.datamaps.klystron import KlystronDataMap, klystron_pvinfo, existing_LCLS_klystrons_sector_station, subbooster_pvinfo, SUBBOOSTER_SECTORS
from pytao import Tao
import json
import os
from lcls_live import data_dir
import pandas as pd

In [2]:
MODEL = 'cu_hxr'
OPTIONS = f'-noplot -slice OTR2:END'
tao = Tao(f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}')

In [3]:

def get_beamcode(model):
    if model == 'cu_hxr':
        beamcode = 1
    elif model == 'cu_sxr':
        beamcode = 2
    else:
        raise ValueError(f'Unknown beam code {beamcode}')        
    
    return beamcode
get_beamcode('cu_sxr')

2

# BPMs

In [4]:
def build_bpm_datamap(tao, model, datum='orbit.x',
                      rate_suffix='1H',
                      factor=.001,
                      ignore_missing_eles = True,
                      check_units_with_epics=False):
    """
    Builds a BPM datamap for setting Tao datums.
    
    BPM signals have attributes of the form: 
        <plane><line><rate>
    where plane is one of:
        X, Y
    line is of the form:
        CUH, CUS, SCH, SCS,
    and rate is one of 
        1H for 1 Hz
        TH for 10 Hz
    
    For example, `:XCUHTH` means
    - horizontal plane `X`
    - Copper Hard line `CUH`
    - Ten Hertz signal `TH`
    
    This attribute will be formed from the model and datum strings.
    
    """ 
    d2, plane = datum.split('.') 

    #
    line = model[0:2].upper() + model[3].upper()
    
    attribute = ':' + plane.upper() + line + rate_suffix
    
    
    df = pd.DataFrame(tao.data_d_array(d2, plane))
    
    if ignore_missing_eles:
        all_eles = tao.lat_list('*', 'ele.name')
        df = df[df['ele_name'].isin(all_eles)]
    
    ixs = df['ix_d1']
    eles = df['ele_name']
    devices = [tao.ele_head(ele)['alias'] for ele in eles]
    pvnames = [d+attribute for d in devices]
    
    # Form output
    df2 = pd.DataFrame()
    
    # Meas
    df2['pvname'] = pvnames
    df2['factor'] = factor
    df2['tao_datum'] =[f"orbit.{plane}[{ix}]|meas" for ix in ixs] 
    
    if check_units_with_epics:
        import epics
        units = epics.caget_many([pv+'.EGU' for pv in pvnames])
        assert len(set(units)) == 1 and 'mm' in units

    dm = TabularDataMap(df2, pvname='pvname', element='tao_datum', factor='factor',
                       tao_format = 'set data {element}  = {value}',
                       bmad_format = '! No equivalent Bmad format for: set data {element}  = {value}'
                       )
    return dm

DM = build_bpm_datamap(tao, MODEL, 'orbit.y')
#DM.pvlist
#DM.pvlist

# Join X and Y
dm = build_bpm_datamap(tao, MODEL, 'orbit.x')
dmy = build_bpm_datamap(tao, MODEL, 'orbit.y')
dm.data = dm.data.append(dmy.data, ignore_index=True)
dm

TabularDataMap(data=                    pvname  factor          tao_datum
0     BPMS:IN20:581:XCUH1H   0.001    orbit.x[7]|meas
1     BPMS:IN20:631:XCUH1H   0.001    orbit.x[8]|meas
2     BPMS:IN20:651:XCUH1H   0.001    orbit.x[9]|meas
3     BPMS:IN20:731:XCUH1H   0.001   orbit.x[10]|meas
4     BPMS:IN20:771:XCUH1H   0.001   orbit.x[11]|meas
..                     ...     ...                ...
333  BPMS:UNDH:5190:YCUH1H   0.001  orbit.y[171]|meas
334   BPMS:DMPH:325:YCUH1H   0.001  orbit.y[172]|meas
335   BPMS:DMPH:381:YCUH1H   0.001  orbit.y[173]|meas
336   BPMS:DMPH:502:YCUH1H   0.001  orbit.y[174]|meas
337   BPMS:DMPH:693:YCUH1H   0.001  orbit.y[175]|meas

[338 rows x 3 columns], pvname='pvname', element='tao_datum', attribute='', factor='factor', offset='', bmad_format='! No equivalent Bmad format for: set data {element}  = {value}', tao_format='set data {element}  = {value}')

## Build datamap for linac

In [5]:
def build_linac_dm(model):
    dat0 = [
        
        {'name': 'BC1_offset',
         'pvname':'BMLN:LI21:235:MOTR',  # mm
         'bmad_factor': 0.001,
         'bmad_name': 'O_BC1_OFFSET',
         'bmad_attribute': 'offset'
        },
        
        {'name': 'BC2_offset',
         'pvname':'BMLN:LI24:805:MOTR', # mm
         'bmad_factor': 0.001,
         'bmad_name': 'O_BC2_OFFSET',
         'bmad_attribute': 'offset'
        },
        
        {
        'name': 'L1_phase',
        'description': 'Controls the L1 phase, which is the single klystron L21_1. We will disable this for now, because the KlystronDataMap handles the phase directly.',
        'pvname': 'ACCL:LI21:1:L1S_S_PV',
        'bmad_name':'O_L1',
        'bmad_factor': 0,  # We'll disable this for now. The Klystron handles it. 
        'bmad_attribute':'phase_deg'
        
    }
    ]
    
    dat_hxr = [
            {
        'name': 'L2_phase',
        'pvname': 'ACCL:LI22:1:PDES',
        'bmad_name':'O_L2',
        'bmad_factor': 1,
        'bmad_attribute':'phase_deg'
        
    },
        {
        'name': 'L3_phase',
        'pvname': 'ACCL:LI25:1:PDES',
        'bmad_name':'O_L3',
        'bmad_attribute':'phase_deg',
        'bmad_offset': 0
        
    }, 
    ]
    
    # SXR has different PVs
    dat_sxr = [
            {
        'name': 'L2_phase',
        'pvname': 'ACCL:LI22:1:PDES:SETDATA_1',
        'bmad_name':'O_L2',
        'bmad_factor': 1,
        'bmad_attribute':'phase_deg'
        
    },
        {
        'name': 'L3_phase',
        'pvname': 'ACCL:LI25:1:PDES:SETDATA_1',
        'bmad_name':'O_L3',
        'bmad_attribute':'phase_deg',
        'bmad_offset': 0
        
    }, 
    ]

    #Note that there are sone NaNs here. That's okay.
    if model == 'cu_hxr':
        df = pd.DataFrame(dat0+dat_hxr)
    elif model == 'cu_sxr':
        df = pd.DataFrame(dat0+dat_sxr)
    else:
        raise ValueError(f'Unknown model: {model}')
        
    dm = TabularDataMap(df, pvname='pvname', element='bmad_name', attribute='bmad_attribute', factor='bmad_factor', offset='bmad_offset')
    
    return dm

## Build datamaps for klystrons

In [6]:
def build_klystron_dms(tao, model):
    
    beamcode = get_beamcode(model)
    
    klystron_names = tao.lat_list('overlay::K*', 'ele.name', flags='-no_slaves')

    klystron_datamaps = []
    for sector, station in existing_LCLS_klystrons_sector_station:
        info = klystron_pvinfo(sector, station, beamcode=beamcode)
        k = KlystronDataMap(**info)

        if k.name in klystron_names:
            klystron_datamaps.append(k)
    
    return klystron_datamaps

## Build datamaps for quadrupoles

In [7]:
def quad_pvinfo(tao, ele):
    """
    Returns dict of PV information for use in a DataMap
    """
    head = tao.ele_head(ele)
    attrs = tao.ele_gen_attribs(ele)
    device = head['alias']
    
    d = {}
    d['bmad_name'] = ele
    d['pvname_rbv'] = device+':BACT'
    d['pvname'] = device+':BDES'    
    d['bmad_factor'] = -1/attrs['L']/10
    d['bmad_attribute'] = 'b1_gradient'
    return d

def build_quad_dm(tao):
    quad_names = tao.lat_list('quad::*', 'ele.name', flags='-no_slaves')
    dfq = pd.DataFrame([quad_pvinfo(tao, ele) for ele in quad_names])
    dm = TabularDataMap(dfq, pvname='pvname', element='bmad_name', attribute = 'bmad_attribute', factor='bmad_factor')
    return dm

## Build datamaps for collimators

In [8]:
CDATA = [('CE11', 'COLL:LI21:235:LVPOS', 'right'),
         ('CE11', 'COLL:LI21:236:LVPOS', 'left')]


def coll_from_dat(bmad_name, pvname, jaw):
    d = {'bmad_name':bmad_name, 'pvname':pvname, 'jaw':jaw}
    if jaw == 'right':
        d['bmad_factor'] = -.001
        d['bmad_attribute'] = 'x1_limit'
    elif jaw == 'left':
        d['bmad_factor'] = .001
        d['bmad_attribute'] = 'x2_limit'
    else:
        raise ValueError(f'Unknown jaw: {jaw}')
        
    return d

def build_collimator_dm(model):
    
    assert model in ['cu_hxr', 'cu_sxr']
    
    df = pd.DataFrame([coll_from_dat(*row) for row in CDATA])
    dm = TabularDataMap(df, pvname='pvname', element='bmad_name', attribute = 'bmad_attribute', factor='bmad_factor')
    return dm
    
build_collimator_dm('cu_hxr')         
        
         
         

TabularDataMap(data=  bmad_name               pvname    jaw  bmad_factor bmad_attribute
0      CE11  COLL:LI21:235:LVPOS  right       -0.001       x1_limit
1      CE11  COLL:LI21:236:LVPOS   left        0.001       x2_limit, pvname='pvname', element='bmad_name', attribute='bmad_attribute', factor='bmad_factor', offset='', bmad_format='{element}[{attribute}] = {value}', tao_format='set ele {element} {attribute} = {value}')

## Build datamap for energy measurements

In [9]:
def build_energy_dm(model):
    # The syntax is flexible enough to use for getting measurements for Tao
    ENERGY_MEAS0 = [
        {
        'name': 'L1_energy',
        'pvname': 'BEND:LI21:231:EDES', # or EDES
        'tao_datum': 'BC1.energy[1]',        
        'factor': 1e9
        },
        {
        'name': 'L2_energy',
        'pvname': 'BEND:LI24:790:EDES', # or EDES
        'tao_datum': 'BC2.energy[1]',
        'factor': 1e9
        }
    ]
    
    
    ENERGY_MEAS_HXR = [ {
        'name': 'L3_HXR_energy',
        'pvname': 'BEND:DMPH:400:EDES', # or EDES
        'tao_datum': 'L3.energy[2]',
        'factor': 1e9
        } ]
    
    ENERGY_MEAS_SXR = [ {
        'name': 'L3_SXR_energy',
        'pvname': 'BEND:DMPS:400:EDES', # or EDES
        'tao_datum': 'L3.energy[2]',
        'factor': 1e9
        } ]    
    
    if model == 'cu_hxr':
        df = pd.DataFrame(ENERGY_MEAS0 + ENERGY_MEAS_HXR)
    elif model == 'cu_sxr':
         df = pd.DataFrame(ENERGY_MEAS0 + ENERGY_MEAS_SXR)
    else:
        raise ValueError(f'Unknown model: {model}')    
    
    
    dm = TabularDataMap(df, pvname='pvname', element='tao_datum', factor='factor',
                       tao_format = 'set data {element}|meas  = {value}',
                       bmad_format = '! No equivalent Bmad format for: set data {element}|meas  = {value}'
                       )
    return dm

## Build subbooster datamap

In [10]:
def build_subbooster_dm(model):
    
    
    beamcode = get_beamcode(model)
    
    subboosters = []
    for sector in SUBBOOSTER_SECTORS:

        dat = subbooster_pvinfo(sector, beamcode=beamcode) 
        dat['bmad_name'] = f'SBST_{sector}'
        dat['bmad_attribute'] = 'phase_deg'
        subboosters.append(dat)
    df = pd.DataFrame(subboosters)    

    # Make the DataMap object, identifying the columns to be used
    dm = TabularDataMap(df, pvname='phase_pvname', element='bmad_name', attribute='bmad_attribute')
    return dm

## Beginning Twiss datamap function

In [11]:
def beginning_meas_twiss_datamap(name, pvprefix):
    dat =  [
    {
    'name': f'{name}_beta_x_meas',
    'pvname': f'{pvprefix}:BETA_X', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'beta_a'
    },
    {
    'name': f'{name}_beta_y_meas',
    'pvname': f'{pvprefix}:BETA_Y', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'beta_b'
    },
    {
    'name': f'{name}_alpha_x_meas',
    'pvname': f'{pvprefix}:ALPHA_X', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'alpha_a'
    },
    {
    'name': f'{name}_alpha_y_meas',
    'pvname': f'{pvprefix}:ALPHA_Y', 
    'bmad_name': 'beginning',   
    'bmad_attribute': 'alpha_b'
    },    
    ]
    
    df= pd.DataFrame(dat)

    return TabularDataMap(df, pvname='pvname', element='bmad_name', attribute = 'bmad_attribute')

## Utility function for creating map for given model 

In [12]:
def build_json_rep(model):
    #set up tao
    # Basic model with options
    MODEL = model
    OPTIONS = f'-slice OTR2:END -noplot'
    INIT = f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}'
    tao = Tao(INIT)
    
    
    rep = []
    
    
    # bpm_dm
    dm = build_bpm_datamap(tao, MODEL, 'orbit.x')
    dmy = build_bpm_datamap(tao, MODEL, 'orbit.y')
    dm.data = dm.data.append(dmy.data, ignore_index=True)
    rep.append(
        {
        "name": "bpms",
        "class": "tabular",
        "data": dm.to_json()
        }
    )        
    
    #collimator_dm
    dm = build_collimator_dm(model)
    rep.append(
        {
        "name": "collimators",
        "class": "tabular",
        "data": dm.to_json()
        }
    )    
    
    #subbooster_dm
    dm = build_subbooster_dm(model)
    rep.append(
        {
        "name": "subboosters",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #linac
    dm = build_linac_dm(model)
    rep.append(
        {
        "name": "linac",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #klystron
    dms = build_klystron_dms(tao, model)
    for dm in dms:
        rep.append(
            {
            "name": dm.name,
            "class": "klystron",
            "data": dm.to_json()
            }
        )
    
    
    #quad
    dm = build_quad_dm(tao)
    rep.append(
        {
        "name": "quad",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    #otr2
    dm = beginning_meas_twiss_datamap('WS02', 'WIRE:IN20:561' )
    rep.append(
        {
        "name": "beginning_WS02",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    
    #energy_meas
    dm = build_energy_dm(model)
    rep.append(
        {
        "name": "tao_energy_measurements",
        "class": "tabular",
        "data": dm.to_json()
        }
    )
    
    return rep

In [13]:
cu_hxr = build_json_rep("cu_hxr")
cu_sxr = build_json_rep("cu_sxr")

## Dump to file

In [14]:
MASTER = {"cu_hxr": cu_hxr, "cu_sxr": cu_sxr}


with open(os.path.join(data_dir, 'datamaps_master.json'), "w") as f:
    json.dump(MASTER, f)